In [ ]:
!pip install faker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime

fake = Faker()

def generate_patient_data(num_records):

    blood_types = ['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-']

    data = []
    for _ in range(num_records):
        first_name = fake.first_name()
        last_name = fake.last_name()
        full_name = f"{first_name} {last_name}"
        dob = fake.date_of_birth(minimum_age=1, maximum_age=100)
        age = (datetime.now().date() - dob).days // 365

        # Generate name-matching email
        email_pattern = random.choice([
            f"{first_name[0]}{last_name}@example.com",
            f"{first_name}.{last_name}@example.com",
            f"{first_name}_{last_name}@example.com",
            f"{first_name}{last_name[:3]}@example.com"
        ]).lower().replace(" ", "")

        record = {
            'Patient id': f"PT{fake.unique.random_number(digits=6)}",
            'Name': full_name,
            'dob': dob.strftime('%Y-%m-%d'),
            'age': age,
            'gender': random.choice(['Male', 'Female', 'Other']),
            'contact no': fake.phone_number(),
            'address': fake.address().replace('\n', ', '),
            'email': email_pattern,
            'blood type': random.choice(blood_types)
        }
        data.append(record)

    return pd.DataFrame(data)

# Generate 15000 patient records (3k for addition)
patients_df = generate_patient_data(15000)

# Display sample
print(patients_df.head())

  Patient id              Name         dob  age gender             contact no  \
0   PT693779   Deborah Stewart  1956-06-24   68  Other  +1-638-606-9750x36048   
1   PT931671  Jessica Chambers  1966-12-17   58   Male          (383)524-7562   
2   PT941739      Nathan White  1984-04-22   41   Male     367-209-1070x78730   
3   PT614528        John Perez  1989-06-23   35   Male     (420)318-0796x5449   
4   PT507807   Jessica Michael  2016-12-28    8  Other     231-453-2908x30516   

                                            address  \
0    3066 Christina Parkways, Burgesstown, AS 90106   
1   07176 Bishop Trace, New Angelaborough, MD 56547   
2        824 Russell Hollow, Courtneyland, SC 98541   
3  6722 Clark Cove Apt. 294, Gonzalesberg, CA 78388   
4   98037 Jones Groves, Lake Matthewmouth, DC 08457   

                          email blood type  
0   deborah_stewart@example.com        AB+  
1  jessica.chambers@example.com        AB+  
2         nathanwhi@example.com         B+  
3 

In [ ]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker()

def generate_doctor_data(num_records):

    positions = [
        "Attending Physician", "Resident", "Fellow",
        "Medical Director", "Chief of Surgery",
        "Consultant", "General Practitioner"
    ]

    data = []
    for _ in range(num_records):
        first_name = fake.first_name()
        last_name = fake.last_name()

        # Generate 8-digit numeric ID (10000000-99999999)
        doctor_id = fake.unique.random_number(digits=8)

        # Professional email pattern
        email = f"{first_name[0].lower()}{last_name.lower()}@hospital.org"

        # Start date (1-40 years experience)
        start_date = fake.date_between(start_date='-40y', end_date='-1m')

        # Position based on experience
        experience = (datetime.now().date() - start_date).days / 365
        if experience > 20:
            position = random.choice(["Medical Director", "Chief of Surgery"])
        elif experience > 10:
            position = "Attending Physician"
        else:
            position = random.choice(["Resident", "Fellow"])

        record = {
            'Doctor Id': doctor_id,  # Pure numeric ID
            'Name': f"Dr. {first_name} {last_name}",
            'Contact No': generate_medical_phone(),
            'Email': email,
            'Position': position,
            'Started At': start_date.strftime('%Y-%m-%d')  # ISO format
        }
        data.append(record)

    return pd.DataFrame(data)

def generate_medical_phone():
    """Generate medical facility-style phone numbers"""
    return f"{random.randint(200, 999)}-{random.randint(200, 999)}-{random.randint(1000, 9999)}"

# Generate 800 doctor records
doctors_df = generate_doctor_data(800)

# Verify uniqueness
print(f"Generated {doctors_df['Doctor Id'].nunique()} unique numeric IDs")

# Display sample
print("\nSample records:")
print(doctors_df.head().to_string(index=False))



Generated 800 unique numeric IDs

Sample records:
 Doctor Id                 Name   Contact No                  Email            Position Started At
  18425458     Dr. Angela Price 573-681-1735    aprice@hospital.org Attending Physician 2014-11-02
  41258173 Dr. Vincent Thornton 201-368-8963 vthornton@hospital.org    Medical Director 1985-09-24
  51601438     Dr. Maureen Yang 420-970-6008     myang@hospital.org Attending Physician 2014-01-31
  55970700 Dr. Kimberly Cordova 676-310-2582  kcordova@hospital.org Attending Physician 2014-01-12
  99041344       Dr. Sara Grant 516-324-5712    sgrant@hospital.org    Medical Director 1989-06-10


In [ ]:
def generate_appointments(num_records, patient_ids, doctor_ids):

    purposes = [
        "Checkup", "Follow-up", "Vaccination", "Pain",
        "Prescription refill", "Test results", "Skin rash",
        "Blood pressure", "Diabetes management", "Cough",
        "Allergies", "Physical", "Headache", "Back pain",
        "Immunization", "Annual exam", "Infection"
    ]

    statuses = [
        "Scheduled", "Completed", "Cancelled", "No-show",
        "Rescheduled"
    ]

    data = []
    for _ in range(num_records):
        # Generate realistic datetime (future appointments more likely)
        appt_date = fake.date_between(start_date='-6m', end_date='+3m')
        appt_time = fake.time(pattern='%H:%M', end_datetime=None)

        # 80% chance status is Completed for past appointments
        if appt_date < datetime.now().date():
            status = "Completed" if random.random() > 0.2 else random.choice(["Cancelled", "No-show"])
        else:
            status = random.choice(["Scheduled", "Rescheduled"])

        data.append({
            'Appointment id': fake.unique.random_number(digits=8),
            'Patient id': random.choice(patient_ids),
            'Doctor id': random.choice(doctor_ids),
            'Appointment date': fake.date_between(start_date='-2y', end_date='today').strftime('%Y-%m-%d'),
            'Appointment time': appt_time,
            'Purpose': random.choice(purposes),
            'Status': status
        })

    return pd.DataFrame(data)

# Generate 7000 appointments (3k extra)
appointments_df = generate_appointments(
    num_records=10000,
    patient_ids=patients_df['Patient id'].tolist(),
    doctor_ids=doctors_df['Doctor Id'].tolist()
)

# Display sample
print("Appointments Sample (10 records):")
print(appointments_df.sample(10).to_string(index=False))



Appointments Sample (10 records):
 Appointment id Patient id  Doctor id Appointment date Appointment time        Purpose    Status
       45180301   PT580545   71697960       2024-02-27            03:09      Infection Completed
       91812620   PT568308   45711466       2024-12-02            02:30      Back pain Completed
       48804842   PT583563    2373324       2025-04-15            07:30      Allergies Completed
       91164447    PT56008   27248260       2024-12-14            22:44   Test results Completed
       11999271   PT280331   10550490       2023-08-28            01:05 Blood pressure Completed
       31534818   PT526716   68661445       2024-12-13            05:05       Headache Completed
       99304848   PT564377   53832475       2023-06-27            20:35   Immunization Completed
       69699301   PT716432   65637149       2024-06-13            03:38      Skin rash Completed
       36300982   PT443130   73294635       2023-07-24            06:38      Back pain Comple

In [ ]:
def generate_prescriptions_from_appointments(appointments_df, max_meds_per_appointment=2):
    """
    Generate prescription records using appointment data.

    Args:
        appointments_df: DataFrame with appointment data including 'Appointment id', 'Patient id', and 'Doctor id'
        max_meds_per_appointment: Maximum number of prescriptions allowed per appointment

    Returns:
        pd.DataFrame with columns:
        - Prescription id
        - Appointment id
        - Patient id
        - Doctor Id
        - Medication name
        - Dosage
        - Frequency
        - Duration (in days)
    """
    # Medication database
    medications = [
        {"name": "Amoxicillin", "dosages": ["250mg", "500mg", "875mg"], "type": "antibiotic", "typical_duration": [7, 10, 14]},
        {"name": "Lisinopril", "dosages": ["5mg", "10mg", "20mg", "40mg"], "type": "hypertension", "typical_duration": [30, 90, 180]},
        {"name": "Atorvastatin", "dosages": ["10mg", "20mg", "40mg", "80mg"], "type": "cholesterol", "typical_duration": [30, 90, 180]},
        {"name": "Metformin", "dosages": ["500mg", "850mg", "1000mg"], "type": "diabetes", "typical_duration": [30, 90, 180]},
        {"name": "Albuterol", "dosages": ["90mcg/inh", "180mcg/inh"], "type": "asthma", "typical_duration": [30, 60]},
        {"name": "Omeprazole", "dosages": ["20mg", "40mg"], "type": "GERD", "typical_duration": [14, 30, 60]},
        {"name": "Sertraline", "dosages": ["25mg", "50mg", "100mg"], "type": "antidepressant", "typical_duration": [30, 90, 180]}
    ]

    frequencies = [
        "Once daily", "Twice daily", "Three times daily",
        "Every 6 hours", "Every 8 hours", "As needed", "Weekly"
    ]

    data = []
    for _, row in appointments_df.iterrows():
        num_meds = random.randint(1, max_meds_per_appointment)
        for _ in range(num_meds):
            med = random.choice(medications)

            # Set duration based on med type
            if med["type"] in ["antibiotic", "GERD"]:
                duration = random.choice(med["typical_duration"])
            else:
                duration = 30 if random.random() > 0.3 else random.choice([60, 90])

            data.append({
                'Prescription id': fake.unique.random_number(digits=8),
                'Appointment id': row['Appointment id'],
                'Patient id': row['Patient id'],
                'Doctor Id': row['Doctor id'],
                'Medication name': med["name"],
                'Dosage': random.choice(med["dosages"]),
                'Frequency': random.choice(frequencies),
                'Duration (days)': duration
            })

    return pd.DataFrame(data)

# Usage example
prescriptions_df = generate_prescriptions_from_appointments(appointments_df)

# Display sample
print("Generated Prescriptions (sample):")
print(prescriptions_df.sample(10).to_string(index=False))


Generated Prescriptions (sample):
 Prescription id  Appointment id Patient id  Doctor Id Medication name Dosage   Frequency  Duration (days)
        16313795        76702791   PT525842   85854839    Atorvastatin   20mg      Weekly               90
        49242189        56402208    PT55582   58821046     Amoxicillin  875mg Twice daily               10
        16328582        19716215   PT754472    9884143      Omeprazole   20mg      Weekly               60
         3678697        47835832   PT395843   66327845      Sertraline  100mg      Weekly               60
        59977011        31162480   PT568207   28798528       Metformin  500mg      Weekly               30
        31408177         7418371    PT53172   77891476      Sertraline  100mg      Weekly               60
        30649206        33327331   PT599076   33273158       Metformin  500mg Twice daily               30
        34647273        68678648   PT420387   54625071      Sertraline   50mg   As needed               30
   

In [ ]:
# Define specialty mapping based on positions
position_to_specialty = {
    # Surgical specialties
    "Chief of Surgery": "General Surgery",
    "Attending Physician": random.choice(["Cardiology", "Internal Medicine", "Gastroenterology"]),
    "Medical Director": random.choice(["Administration", "Internal Medicine"]),
    "Consultant": lambda: random.choice(["Neurology", "Oncology", "Endocrinology"]),
    "General Practitioner": "Family Medicine",
    "Resident": lambda: random.choice([
        "Internal Medicine Resident",
        "Surgery Resident",
        "Pediatrics Resident"
    ]),
    "Fellow": lambda: random.choice([
        "Cardiology Fellow",
        "Oncology Fellow",
        "Pulmonary Fellow"
    ])
}

def determine_specialty(position):
    """Determine specialty based on position with realistic logic"""
    specialty = position_to_specialty.get(position)

    # Handle callable specialties (dynamic choices)
    if callable(specialty):
        return specialty()
    return specialty

# Add specialty column to existing DataFrame
doctors_df['Specialty'] = doctors_df['Position'].apply(determine_specialty)

# Display sample
print("\nDoctors with Specialties:")
print(doctors_df[['Doctor Id', 'Position', 'Specialty']].sample(10).to_string(index=False))


Doctors with Specialties:
 Doctor Id            Position                  Specialty
  16256112    Medical Director             Administration
  99623969              Fellow          Cardiology Fellow
  65014330    Medical Director             Administration
  42097235 Attending Physician          Internal Medicine
  66407041            Resident           Surgery Resident
  21844372 Attending Physician          Internal Medicine
    691283            Resident Internal Medicine Resident
  83800810    Medical Director             Administration
  27797164 Attending Physician          Internal Medicine
  56167194    Medical Director             Administration


In [ ]:
def generate_medicines(num_records):

    # Common medicine types and their brands
    medicine_data = [
        {"name": "Amoxicillin", "type": "Antibiotic", "brands": ["Amoxil", "Trimox", "Moxatag"],
         "dosages": ["250mg", "500mg", "875mg"]},
        {"name": "Lisinopril", "type": "ACE Inhibitor", "brands": ["Zestril", "Prinivil", "Qbrelis"],
         "dosages": ["5mg", "10mg", "20mg", "40mg"]},
        {"name": "Atorvastatin", "type": "Statin", "brands": ["Lipitor", "Atorva", "Torvast"],
         "dosages": ["10mg", "20mg", "40mg", "80mg"]},
        {"name": "Metformin", "type": "Biguanide", "brands": ["Glucophage", "Fortamet", "Glumetza"],
         "dosages": ["500mg", "850mg", "1000mg"]},
        {"name": "Albuterol", "type": "Bronchodilator", "brands": ["ProAir", "Ventolin", "Proventil"],
         "dosages": ["90mcg/inh", "180mcg/inh"]},
        {"name": "Omeprazole", "type": "PPI", "brands": ["Prilosec", "Losec", "Zegerid"],
         "dosages": ["20mg", "40mg"]},
        {"name": "Sertraline", "type": "SSRI", "brands": ["Zoloft", "Lustral", "Asentra"],
         "dosages": ["25mg", "50mg", "100mg"]}
    ]

    data = []
    for _ in range(num_records):
        med = random.choice(medicine_data)
        brand = random.choice(med["brands"])
        dosage = random.choice(med["dosages"])

        # Generate realistic expiry date (1-3 years from now)
        expiry_date = fake.date_between(start_date='today', end_date='+3y')

        data.append({
            'Medicine id': fake.unique.random_number(digits=8),
            'Name': med["name"],
            'Brand': brand,
            'Type': med["type"],
            'Dosage': dosage,
            'Stock quantity': random.randint(0, 500),  # Some may be out of stock
            'Expiry date': expiry_date.strftime('%Y-%m-%d')
        })

    return pd.DataFrame(data)

# Generate 800 medicine records
medicines_df = generate_medicines(800)

# Display sample
print("Medicines Sample (10 records):")
print(medicines_df.sample(10).to_string(index=False))


Medicines Sample (10 records):
 Medicine id         Name    Brand          Type Dosage  Stock quantity Expiry date
    17613617    Metformin Glumetza     Biguanide  850mg              87  2028-01-28
    93804937    Metformin Fortamet     Biguanide 1000mg             478  2028-03-21
     4740164   Omeprazole Prilosec           PPI   40mg             448  2026-09-03
    93838863   Omeprazole Prilosec           PPI   40mg             228  2025-05-18
    32801884   Omeprazole  Zegerid           PPI   40mg             370  2028-01-28
    24135250 Atorvastatin   Atorva        Statin   40mg              71  2027-02-15
    34437076  Amoxicillin   Trimox    Antibiotic  250mg             146  2026-07-15
    49750808  Amoxicillin   Amoxil    Antibiotic  250mg             134  2028-04-02
    20847594   Lisinopril  Qbrelis ACE Inhibitor   10mg             346  2026-09-30
    61352760  Amoxicillin  Moxatag    Antibiotic  875mg             419  2027-08-14


In [ ]:
def generate_ambulances(num_records):

    # US ambulance numbering patterns (state abbreviation + number)
    states = ['NY', 'CA', 'TX', 'FL', 'IL', 'PA', 'OH', 'GA', 'NC', 'MI']

    data = []
    for _ in range(num_records):
        # Generate realistic ambulance number (e.g., NY-AMB-1072)
        ambulance_number = f"{random.choice(states)}-AMB-{random.randint(1000, 9999)}"

        # Acquisition date (1-10 years ago)
        acquisition_date = fake.date_between(start_date='-10y', end_date='-1y')

        # Last service date (between acquisition and today)
        last_service_date = fake.date_between(start_date=acquisition_date, end_date='today')

        # Availability based on service recency (serviced in last month = available)
        is_available = (datetime.now().date() - last_service_date).days < 30

        data.append({
            'Ambulance id': fake.unique.random_number(digits=6),  # Shorter ID for vehicles
            'Ambulance number': ambulance_number,
            'Availability': "Available" if is_available else "In Maintenance",
            'Last service date': last_service_date.strftime('%Y-%m-%d'),
            'Acquisition date': acquisition_date.strftime('%Y-%m-%d')
        })

    return pd.DataFrame(data)

# Generate 200 ambulance records
ambulances_df = generate_ambulances(200)

# Display sample
print("Ambulances Sample (10 records):")
print(ambulances_df.sample(10).to_string(index=False))


Ambulances Sample (10 records):
 Ambulance id Ambulance number   Availability Last service date Acquisition date
       229108      NY-AMB-7428 In Maintenance        2024-10-04       2020-08-27
       135557      NY-AMB-4328 In Maintenance        2022-01-16       2018-02-02
       679218      NY-AMB-9162 In Maintenance        2022-03-19       2017-06-23
       915654      CA-AMB-4672 In Maintenance        2021-03-01       2020-12-30
       519967      MI-AMB-6766 In Maintenance        2016-02-03       2015-09-14
       102369      MI-AMB-3367 In Maintenance        2021-10-14       2019-11-20
       313413      IL-AMB-4946 In Maintenance        2022-06-07       2015-09-06
       785358      OH-AMB-8508 In Maintenance        2024-09-08       2023-05-01
       823177      IL-AMB-8330 In Maintenance        2022-03-17       2016-09-23
       977724      NY-AMB-9534 In Maintenance        2022-04-12       2019-04-25


In [ ]:
def generate_ambulance_logs(num_records, ambulance_ids, patient_ids):

    statuses = [
        "Completed",
        "In Progress",
        "Cancelled",
        "Patient Refused",
        "Diverted"
    ]

    # Common hospital names for dropoff locations
    hospitals = [
        "General Hospital",
        "City Medical Center",
        "University Hospital",
        "Children's Hospital",
        "Regional Trauma Center"
    ]

    data = []
    for _ in range(num_records):
        # Generate realistic time intervals (pickup to dropoff 15-90 mins)
        pickup_time = fake.date_time_between(start_date='-1y', end_date='now')
        dropoff_time = pickup_time + timedelta(minutes=random.randint(15, 90))

        # Get random hospital and add address
        hospital = random.choice(hospitals)
        dropoff_location = f"{hospital}, {fake.street_address()}, {fake.city()}"

        # 10% chance of cancelled/diverted transports
        if random.random() < 0.1:
            status = random.choice(["Cancelled", "Patient Refused", "Diverted"])
            dropoff_time = None
            dropoff_location = "N/A"
        else:
            status = "Completed" if dropoff_time < datetime.now() else "In Progress"

        data.append({
            'Log_id': fake.unique.random_number(digits=8),
            'Ambulance id': random.choice(ambulance_ids),
            'Patient id': random.choice(patient_ids),
            'Pickup location': fake.street_address() + ", " + fake.city(),
            'Dropoff location': dropoff_location,
            'Pickup time': pickup_time.strftime('%Y-%m-%d %H:%M:%S'),
            'Dropoff time': dropoff_time.strftime('%Y-%m-%d %H:%M:%S') if dropoff_time else "N/A",
            'Status': status
        })

    return pd.DataFrame(data)

# Generate 600 ambulance logs
ambulance_logs_df = generate_ambulance_logs(
    num_records=600,
    ambulance_ids=ambulances_df['Ambulance id'].tolist(),
    patient_ids=patients_df['Patient id'].tolist()
)

# Display sample
print("Ambulance Logs Sample (10 records):")
print(ambulance_logs_df.sample(10).to_string(index=False))

Ambulance Logs Sample (10 records):
  Log_id  Ambulance id Patient id                             Pickup location                                               Dropoff location         Pickup time        Dropoff time          Status
41641775         65114   PT642447             08635 Kim Summit, Jenniferhaven       City Medical Center, 074 Katelyn Ville, New Melanieburgh 2024-09-16 21:46:07 2024-09-16 22:54:07       Completed
 4841917        262489   PT622514    89522 Welch Estate Suite 618, Lozanoport               General Hospital, 01485 Hannah Lane, Josephmouth 2025-04-08 16:26:13 2025-04-08 17:53:13       Completed
61878751        436379   PT789374 6377 Smith Islands Suite 505, Port Kimberly                                                            N/A 2024-11-24 01:29:09                 N/A        Diverted
92899967        647468   PT729702              0179 Smith Centers, West Brian        University Hospital, 3263 Jonathan Haven, Mitchellhaven 2024-10-26 20:57:56 2024-10-26 21:5

In [ ]:
def generate_rooms(num_records):

    room_types = [
        ("Emergency", 1),
        ("Operating", 1),
        ("ICU", 1),
        ("CCU", 1),
        ("Maternity", 2),
        ("Pediatric", 2),
        ("General Ward", 4),
        ("Isolation", 1),
        ("Recovery", 2),
        ("Psychiatric", 1)
    ]

    building_floors = ['A', 'B', 'C', 'D']

    data = []
    for _ in range(num_records):
        room_type, capacity = random.choice(room_types)
        floor = random.randint(1, 10)
        building = random.choice(building_floors)
        room_num = f"{building}{floor}{random.randint(10, 99)}"

        # Last serviced date (more frequent for critical care rooms)
        if room_type in ["ICU", "CCU", "Operating"]:
            last_service = fake.date_between(start_date='-1w', end_date='today')
        else:
            last_service = fake.date_between(start_date='-1m', end_date='today')

        # Status based on service date and room type
        days_since_service = (datetime.now().date() - last_service).days
        if days_since_service > 7:
            status = "Needs Cleaning"
        elif room_type in ["ICU", "CCU"] and days_since_service > 2:
            status = "Needs Inspection"
        else:
            status = "Ready"

        data.append({
            'Room id': fake.unique.random_number(digits=6),
            'Room number': room_num,
            'Room type': room_type,
            'Capacity': capacity,
            'Status': status,
            'Last serviced': last_service.strftime('%Y-%m-%d')
        })

    return pd.DataFrame(data)

# Generate 200 room records
rooms_df = generate_rooms(200)

# Display sample
print("Hospital Rooms Sample (10 records):")
print(rooms_df.sample(10).to_string(index=False))

Hospital Rooms Sample (10 records):
 Room id Room number    Room type  Capacity           Status Last serviced
  820202        B937     Recovery         2            Ready    2025-05-17
  874884        B636 General Ward         4            Ready    2025-05-17
  649182        D739    Isolation         1            Ready    2025-05-17
  481598        A821          CCU         1 Needs Inspection    2025-05-11
  874451        A146    Maternity         2            Ready    2025-05-17
  379828        B665          ICU         1 Needs Inspection    2025-05-14
  742898        A317    Emergency         1            Ready    2025-05-17
  388036        B122          ICU         1 Needs Inspection    2025-05-15
  583638        C752     Recovery         2            Ready    2025-05-17
  972033        B648    Pediatric         2            Ready    2025-05-17


In [ ]:
def generate_medical_records_from_appointments(appointments_df, max_records_per_appointment=1):
    """
    Generate medical records with at least one entry per appointment using patient and doctor info from appointments.

    Args:
        appointments_df: DataFrame containing 'Appointment id', 'Patient id', and 'Doctor id'
        max_records_per_appointment: Max number of medical records per appointment

    Returns:
        DataFrame with:
        - Record id
        - Appointment id
        - Patient id
        - Doctor id
        - Diagnosis
        - Treatment
    """
    # Common diagnoses with ICD-10 codes
    diagnoses = [
        ("Hypertension", "I10"),
        ("Type 2 Diabetes", "E11.9"),
        ("Acute Bronchitis", "J20.9"),
        ("Major Depressive Disorder", "F32.9"),
        ("COVID-19", "U07.1"),
        ("Asthma", "J45.909"),
        ("Back Pain", "M54.5"),
        ("Migraine", "G43.909"),
        ("UTI", "N39.0"),
        ("Hyperlipidemia", "E78.5")
    ]

    treatments = [
        "Prescribed medication",
        "Physical therapy",
        "Surgery scheduled",
        "Lifestyle modification",
        "Referral to specialist",
        "Monitoring recommended",
        "IV fluids administered",
        "Wound care",
        "Immunization given",
        "Observation only"
    ]

    data = []
    for _, row in appointments_df.iterrows():
        num_records = random.randint(1, max_records_per_appointment)
        for _ in range(num_records):
            diagnosis, icd10 = random.choice(diagnoses)

            # Custom treatments for specific diagnoses
            if diagnosis == "Hypertension":
                treatment = random.choice([
                    "Prescribed Lisinopril 10mg daily",
                    "Recommended low-sodium diet",
                    "BP monitoring scheduled"
                ])
            elif diagnosis == "Type 2 Diabetes":
                treatment = random.choice([
                    "Prescribed Metformin 500mg twice daily",
                    "Referred to endocrinologist",
                    "Diabetes education ordered"
                ])
            else:
                treatment = random.choice(treatments)

            data.append({
                'Record id': fake.unique.random_number(digits=8),
                'Appointment id': row['Appointment id'],
                'Patient id': row['Patient id'],
                'Doctor id': row['Doctor id'],
                'Diagnosis': f"{diagnosis} ({icd10})",
                'Treatment': treatment
            })

    return pd.DataFrame(data)

# Example usage:
medical_records_df = generate_medical_records_from_appointments(
    appointments_df,
    max_records_per_appointment=1  # you can increase this for richer data
)

# Display sample
print("Medical Records Sample (10 records):")
print(medical_records_df.sample(10).to_string(index=False))


Medical Records Sample (10 records):
 Record id  Appointment id Patient id  Doctor id                Diagnosis                              Treatment
  67727581         4589809   PT630394   38332606  Type 2 Diabetes (E11.9) Prescribed Metformin 500mg twice daily
  94575913        30005571   PT218352    9884143   Hyperlipidemia (E78.5)                       Physical therapy
  69574939        68820065   PT271685   23185626       Migraine (G43.909)                 Lifestyle modification
  98986563        27881934   PT549676   35705256        Back Pain (M54.5)                 IV fluids administered
  28948471        25969952   PT175165   81108815 Acute Bronchitis (J20.9)                             Wound care
  31138152        91485351   PT319639   55673038  Type 2 Diabetes (E11.9)             Diabetes education ordered
  78367461        53363685   PT562160   54563499        Back Pain (M54.5)                       Observation only
  53917902        68343159   PT172109   54756090   Hyperlip

In [ ]:
def generate_pharmacy_data_from_prescriptions(prescriptions_df, medicines_df):
    """
    Generate pharmacy records based on prescriptions.

    Each row corresponds to one prescription, linking to a medicine in medicines_df
    by matching the name and dosage.

    Returns:
        DataFrame with:
        - Pharmacy id
        - Prescription id
        - Medicine id
        - Patient id
        - Quantity
    """
    data = []

    for _, row in prescriptions_df.iterrows():
        med_name = row['Medication name']
        dosage = row['Dosage']

        # Match medicine name and dosage in medicines_df
        match = medicines_df[
            (medicines_df['Name'] == med_name) &
            (medicines_df['Dosage'] == dosage)
        ]

        # Only proceed if a valid match exists
        if not match.empty:
            medicine_id = match.iloc[0]['Medicine id']
        else:
            continue  # skip if no matching medicine found

        data.append({
            'Pharmacy id': fake.unique.random_number(digits=8),
            'Prescription id': row['Prescription id'],
            'Medicine id': medicine_id,
            'Patient id': row['Patient id'],
            'Quantity': random.randint(1, 5)
        })

    return pd.DataFrame(data)

# Example usage:
pharmacy_df = generate_pharmacy_data_from_prescriptions(prescriptions_df, medicines_df)

# Display sample
print("Pharmacy Records Sample (10 records):")
print(pharmacy_df.sample(10).to_string(index=False))


Pharmacy Records Sample (10 records):
 Pharmacy id  Prescription id  Medicine id Patient id  Quantity
    80880674         62792847     14872313   PT365886         5
    31035980         45282717     64695009   PT555283         1
    78005503         49508751     12812185   PT790003         1
    27158609         30133804     96327970   PT559278         2
    44697017         24680874     52033297   PT358765         5
    32999065         97240999      7156221   PT877177         5
    55042269          3015713      7156221   PT937651         5
    37995412         15898642     96327970   PT136419         4
    60064989         21334426     88786614   PT131167         4
    33570959         96344127     88786614   PT958723         2


In [ ]:
pharmacy_df.shape

(14942, 5)

In [ ]:
import numpy as np
medicines_df['Price'] = np.round(np.random.uniform(5, 200, size=len(medicines_df)), 2)

# Display sample
print("Medicines with Prices (10 records):")
print(medicines_df.sample(10).to_string(index=False))

Medicines with Prices (10 records):
 Medicine id         Name    Brand           Type    Dosage  Stock quantity Expiry date  Price
    40546141  Amoxicillin   Amoxil     Antibiotic     500mg              23  2025-08-17 150.33
    61794136 Atorvastatin  Lipitor         Statin      40mg             225  2025-08-11 112.50
    26741517   Sertraline   Zoloft           SSRI      25mg             173  2026-07-02  16.18
    82015767 Atorvastatin  Lipitor         Statin      10mg              13  2027-07-21  43.09
    44931128   Sertraline  Asentra           SSRI     100mg              54  2027-01-27 122.55
    20167663    Metformin Fortamet      Biguanide     500mg              73  2027-04-23 108.87
     2922860    Metformin Glumetza      Biguanide     850mg             381  2027-08-02  34.70
    89723297    Albuterol Ventolin Bronchodilator 90mcg/inh             283  2028-01-12 118.66
    46801947  Amoxicillin   Amoxil     Antibiotic     250mg             205  2025-07-17 102.82
    97046764  

In [ ]:
from datetime import datetime, timedelta



def generate_room_assignments(num_records, room_ids, patient_ids):
    data = []

    for _ in range(num_records):
        assignment_date = fake.date_between(
            start_date=datetime(2023, 1, 1).date(),
            end_date=datetime(2025, 4, 1).date()
        )

        end_date = assignment_date + timedelta(days=random.randint(1, 15))

        data.append({
            'Assignment id': fake.unique.random_number(digits=8),
            'Room id': random.choice(room_ids),
            'Patient id': random.choice(patient_ids),
            'Assignment date': assignment_date,
            'End date': end_date
        })

    return pd.DataFrame(data)


# Example usage
room_assignments_df = generate_room_assignments(
    num_records=7000,
    room_ids=rooms_df['Room id'].tolist(),
    patient_ids=patients_df['Patient id'].tolist()
)

# Display sample
print("Room Assignments Sample (10 records):")
print(room_assignments_df.sample(10).to_string(index=False))


Room Assignments Sample (10 records):
 Assignment id  Room id Patient id Assignment date   End date
      95255623   905956   PT464533      2024-05-31 2024-06-04
       5204290   747042   PT367604      2025-03-15 2025-03-25
      12583238   244485   PT830828      2023-06-05 2023-06-16
      98687503   615794   PT200842      2023-02-28 2023-03-07
      17059518   998471   PT302346      2023-03-10 2023-03-25
      13168882   220565   PT391752      2025-03-04 2025-03-17
      54956773   149752   PT190320      2024-07-12 2024-07-19
      63763803   315025   PT394654      2024-10-21 2024-10-30
        375325   483067   PT813267      2024-03-27 2024-04-03
       9761828   735461   PT833907      2023-05-02 2023-05-12


In [ ]:
room_assignments_df.shape

(7000, 5)

In [ ]:
def generate_medical_record_medicines_linked(
    medical_records_df,
    prescriptions_df,
    pharmacy_df
):
    """
    Generate medical record medicines table using pharmacy and prescription data.

    Columns:
    - MRM id (unique)
    - Record id (from medical_records_df)
    - Medicine id (from pharmacy_df)
    - Dosage (from prescriptions_df)
    """
    data = []

    for _, record in medical_records_df.iterrows():
        record_id = record['Record id']
        appointment_id = record['Appointment id']

        # Skip if appointment_id is None (e.g., walk-ins)
        if pd.isna(appointment_id):
            continue

        # Get all prescriptions for this appointment
        matching_prescriptions = prescriptions_df[prescriptions_df['Appointment id'] == appointment_id]

        for _, pres in matching_prescriptions.iterrows():
            prescription_id = pres['Prescription id']
            dosage = pres['Dosage']

            # Get all pharmacy records for this prescription
            matching_pharmacy = pharmacy_df[pharmacy_df['Prescription id'] == prescription_id]

            for _, pharm in matching_pharmacy.iterrows():
                medicine_id = pharm['Medicine id']

                data.append({
                    'MRM id': fake.unique.random_number(digits=8),
                    'Record id': record_id,
                    'Medicine id': medicine_id,
                    'Dosage': dosage
                })

    return pd.DataFrame(data)

# Example usage
medical_record_medicine_df = generate_medical_record_medicines_linked(
    medical_records_df=medical_records_df,
    prescriptions_df=prescriptions_df,
    pharmacy_df=pharmacy_df
)

# Display sample
print("Medical Record Medicines Sample (10 records):")
print(medical_record_medicine_df.sample(10).to_string(index=False))


Medical Record Medicines Sample (10 records):
  MRM id  Record id  Medicine id Dosage
52475718   49190602     88786614  250mg
99115322   21693162     88786614  250mg
63206431   59963582     35984591   25mg
86860087   61887893     35984591   25mg
 4958496   31194493     88786614  250mg
37883402   68727748      2098773   10mg
29224272   67750715     24135250   40mg
69341552   83415645      7156221  100mg
79724517   41194947     38738005 1000mg
91380591   37418826     79819134   50mg


In [ ]:
medical_record_medicine_df.shape

(14942, 4)

Tables to regenerate: prescriptions, medical records, pharmacy, medical record medicines, billing


In [ ]:
def generate_billing_data_from_appointments(appointments_df):
    insurance_providers = [
        "Aetna", "UnitedHealthcare", "Blue Cross Blue Shield",
        "Cigna", "Kaiser Permanente", "None"
    ]
    payment_statuses = ["Paid", "Pending", "Denied", "Partially Paid"]

    billing_data = []

    for _, row in appointments_df.iterrows():
        appointment_id = row['Appointment id']
        patient_id = row['Patient id']
        appointment_date = pd.to_datetime(row['Appointment date'])

        billing_data.append({
            'Bill id': fake.unique.random_number(digits=8),
            'Patient id': patient_id,
            'Appointment id': appointment_id,
            'Total amount': round(random.uniform(100, 1000), 2),
            'Payment status': random.choice(payment_statuses),
            'Payment date': appointment_date.strftime('%Y-%m-%d'),
            'Insurance provider': random.choice(insurance_providers)
        })

    return pd.DataFrame(billing_data)

# Example usage
billing_df = generate_billing_data_from_appointments(appointments_df)

# Sample output
print("Billing Records Sample (10 records):")
print(billing_df.sample(10).to_string(index=False))


Billing Records Sample (10 records):
 Bill id Patient id  Appointment id  Total amount Payment status Payment date     Insurance provider
95426589     PT7971        59088453        885.90         Denied   2024-05-18 Blue Cross Blue Shield
75943081   PT183500         4746702        376.31         Denied   2024-04-09                   None
 7965735   PT759001        84754488        871.60        Pending   2024-03-27                   None
93696559   PT132476        28176147        150.94        Pending   2025-03-28       UnitedHealthcare
30802704    PT38482        44436231        742.72           Paid   2023-11-13                  Cigna
13929426   PT743524        60951099        210.77         Denied   2025-01-28      Kaiser Permanente
94391576   PT353802        54082605        119.05         Denied   2024-01-11      Kaiser Permanente
73211405    PT50152        75485778        555.87         Denied   2023-11-24                  Cigna
49335948    PT91759        63893077        799.73     

In [ ]:
prescriptions_df.shape

(14942, 8)

In [ ]:
medical_records_df.shape

(10000, 6)

In [ ]:
billing_df.shape

(10000, 7)

In [ ]:
import pandas as pd
import random
import numpy as np
import string
from datetime import datetime

def inject_data_issues(df, issues_config):
    """
    Injects data quality issues into a copy of the DataFrame.

    issues_config: {
        'missing_values': {'columns': [...], 'percentage': float},
        'date_format_issues': {'columns': [...], 'percentage': float},
        'duplicates': {'percentage': float},
        'text_corruption': {'columns': [...], 'percentage': float}
    }
    """
    dirty_df = df.copy()
    total_rows = len(dirty_df)

    # Inject missing values
    if 'missing_values' in issues_config:
        columns = issues_config['missing_values']['columns']
        pct = issues_config['missing_values']['percentage']
        n = int(pct * total_rows)

        for col in columns:
            indices = random.sample(range(total_rows), n)
            dirty_df.loc[indices, col] = np.nan

    # Inject inconsistent date formats
    if 'date_format_issues' in issues_config:
        columns = issues_config['date_format_issues']['columns']
        pct = issues_config['date_format_issues']['percentage']
        n = int(pct * total_rows)

        for col in columns:
            indices = random.sample(range(total_rows), n)
            for idx in indices:
                original_date = dirty_df.loc[idx, col]
                if pd.isnull(original_date):
                    continue
                try:
                    date_obj = pd.to_datetime(original_date)
                    dirty_df.loc[idx, col] = random.choice([
                        date_obj.strftime('%d-%m-%Y'),
                        date_obj.strftime('%m/%d/%Y'),
                        date_obj.strftime('%b %d, %Y'),
                        date_obj.strftime('%Y.%m.%d'),
                    ])
                except Exception:
                    continue

    # Inject duplicate rows
    if 'duplicates' in issues_config:
        pct = issues_config['duplicates']['percentage']
        n = int(pct * total_rows)
        dup_rows = dirty_df.sample(n=n, replace=False)
        dirty_df = pd.concat([dirty_df, dup_rows], ignore_index=True)

    # Inject text corruption (special characters)
    if 'text_corruption' in issues_config:
        columns = issues_config['text_corruption']['columns']
        pct = issues_config['text_corruption']['percentage']
        n = int(pct * total_rows)
        special_chars = list("!@#$%^&*()_+=[]{}|:;<>,.?/~`")

        for col in columns:
            indices = random.sample(range(total_rows), n)
            for idx in indices:
                val = dirty_df.loc[idx, col]
                if pd.isnull(val) or not isinstance(val, str):
                    continue
                insert_pos = random.randint(0, len(val))
                num_chars = random.randint(1, 3)
                corruption = ''.join(random.choices(special_chars, k=num_chars))
                corrupted_val = val[:insert_pos] + corruption + val[insert_pos:]
                dirty_df.loc[idx, col] = corrupted_val

    return dirty_df


In [ ]:
medical_records_df.head()

,Record id,Appointment id,Patient id,Doctor id,Diagnosis,Treatment
0,70626890,68675075,PT81957,37309933,Hyperlipidemia (E78.5),Referral to specialist
1,17836511,16171161,PT561107,76652128,Acute Bronchitis (J20.9),Prescribed medication
2,77646316,60345092,PT633354,38353840,Acute Bronchitis (J20.9),Surgery scheduled
3,46106160,53346682,PT339587,57477748,Acute Bronchitis (J20.9),Referral to specialist
4,92900832,99453278,PT387755,35255412,Type 2 Diabetes (E11.9),Referred to endocrinologist


In [ ]:
dirty_medical_records_df = inject_data_issues(
    df=medical_records_df,
    issues_config={
        'missing_values': {'columns': ['Treatment', 'Diagnosis'], 'percentage': 0.2},
        'duplicates': {'percentage': 0.05},
        'text_corruption': {'columns': ['Diagnosis', 'Treatment'], 'percentage': 0.1}
    }
)

print(dirty_medical_records_df.sample(10).to_string(index=False))


 Record id  Appointment id Patient id  Doctor id                         Diagnosis                   Treatment
  87958594        74295796   PT159002   45722988            Hyperlipidemia (E78.5)        Immun$;ization given
  60661921        56605804   PT710567   24318692                Hypertension (I10)     BP monitoring scheduled
  11484927        47559110   PT757168   92045276           Type 2 Diabetes (E11.9) Referred to endocrinologist
  16290364        30951515   PT596704   31612415                  COVID-19 (U07.1)           Surgery scheduled
  39805762        94733170   PT913429   38778819           Type 2 Diabetes (E11.9)  Diabetes education ordered
  25544218        45937709   PT390585   92455433          Acute Bronchitis (J20.9)       Prescribed medication
  70687332        62626661   PT708080   21609602                               NaN                  Wound care
  31677435        96670342   PT129294   21609602 Major Depressive Disorder (F32.9)      Lifestyle modification
 

In [ ]:
patients_df.head()

,Patient id,Name,dob,age,gender,contact no,address,email,blood type
0,PT693779,Deborah Stewart,1956-06-24,68,Other,+1-638-606-9750x36048,"3066 Christina Parkways, Burgesstown, AS 90106",deborah_stewart@example.com,AB+
1,PT931671,Jessica Chambers,1966-12-17,58,Male,(383)524-7562,"07176 Bishop Trace, New Angelaborough, MD 56547",jessica.chambers@example.com,AB+
2,PT941739,Nathan White,1984-04-22,41,Male,367-209-1070x78730,"824 Russell Hollow, Courtneyland, SC 98541",nathanwhi@example.com,B+
3,PT614528,John Perez,1989-06-23,35,Male,(420)318-0796x5449,"6722 Clark Cove Apt. 294, Gonzalesberg, CA 78388",john_perez@example.com,B+
4,PT507807,Jessica Michael,2016-12-28,8,Other,231-453-2908x30516,"98037 Jones Groves, Lake Matthewmouth, DC 08457",jessica.michael@example.com,B+


In [ ]:
doctors_df.head()

,Doctor Id,Name,Contact No,Email,Position,Started At,Specialty
0,18425458,Dr. Angela Price,573-681-1735,aprice@hospital.org,Attending Physician,2014-11-02,Internal Medicine
1,41258173,Dr. Vincent Thornton,201-368-8963,vthornton@hospital.org,Medical Director,1985-09-24,Administration
2,51601438,Dr. Maureen Yang,420-970-6008,myang@hospital.org,Attending Physician,2014-01-31,Internal Medicine
3,55970700,Dr. Kimberly Cordova,676-310-2582,kcordova@hospital.org,Attending Physician,2014-01-12,Internal Medicine
4,99041344,Dr. Sara Grant,516-324-5712,sgrant@hospital.org,Medical Director,1989-06-10,Administration


In [ ]:
dirty_doctors_df = inject_data_issues(
    df=doctors_df,
    issues_config={
        'missing_values': {'columns': ['Contact No', 'Email', 'Position'], 'percentage': 0.10},
        'date_format_issues': {'columns': ['Started At'], 'percentage': 0.10},
        'duplicates': {'percentage': 0.09},
        'text_corruption': {'columns': ['Position'], 'percentage': 0.13}
    }
)

print(dirty_doctors_df.sample(10).to_string(index=False))


 Doctor Id               Name   Contact No                 Email               Position   Started At                  Specialty
  36183974  Dr. Steven Werner          NaN  swerner@hospital.org       Chief of Surgery   1988-10-18            General Surgery
  96655296     Dr. Sara Evans 887-427-8383   sevans@hospital.org    Attending Physician   2012-01-19          Internal Medicine
  16735423 Dr. Kimberly Walls 694-209-2120                   NaN       Medical Director   1985-09-08             Administration
   5847082   Dr. Jared Howell 596-378-5407  jhowell@hospital.org                    NaN   1992-10-23             Administration
  38714427  Dr. David Johnson 642-629-1312                   NaN    Attending Physician   2014-12-26          Internal Medicine
  71432333  Dr. Leslie Rogers 748-898-6097  lrogers@hospital.org       Chief of Surgery   1996-04-18            General Surgery
  43141590     Dr. Robert Cox 990-359-7090     rcox@hospital.org    Attending Physician   2007-08-14    

In [ ]:
prescriptions_df.head()

,Prescription id,Appointment id,Patient id,Doctor Id,Medication name,Dosage,Frequency,Duration (days)
0,2455746,68675075,PT81957,37309933,Albuterol,90mcg/inh,Twice daily,90
1,33763380,16171161,PT561107,76652128,Atorvastatin,20mg,Twice daily,30
2,31089587,16171161,PT561107,76652128,Amoxicillin,875mg,Every 6 hours,10
3,3198692,60345092,PT633354,38353840,Metformin,500mg,Once daily,30
4,16900158,53346682,PT339587,57477748,Metformin,500mg,Every 8 hours,30


In [ ]:
dirty_prescriptions_df = inject_data_issues(
    df=prescriptions_df,
    issues_config={
        'missing_values': {'columns': ['Medication name', 'Dosage', 'Frequency'], 'percentage': 0.15},
        'duplicates': {'percentage': 0.25},
        'text_corruption': {'columns': ['Medication name', 'Frequency'], 'percentage': 0.15}
    }
)

print(dirty_prescriptions_df.sample(10).to_string(index=False))


 Prescription id  Appointment id Patient id  Doctor Id Medication name     Dosage     Frequency  Duration (days)
        42572870        48053694   PT714113   10312486    Atorvastatin        NaN    Once daily               90
          408838        53416275   PT378820   52567165     Amoxicillin      250mg    Once daily               10
        19287598        93501451   PT644301   73871464       Metformin     1000mg           NaN               90
         6517929        35778400    PT97408   17185463       Metformin      500mg        Weekly               60
        93913821        77036831   PT708999   84006839       Albuterol 180mcg/inh    Once daily               90
        71769916         2759227   PT778133   21198986    Atorvastatin       40mg Every 6 hours               30
        24510858        66895689   PT412572   10700192       Metformin      500mg        Weekly               30
        77979468         6750781   PT305952   35887951    Sert@>raline      100mg    Once daily 

In [ ]:
dirty_prescriptions_df.isnull().sum()

,0
Prescription id,0
Appointment id,0
Patient id,0
Doctor Id,0
Medication name,2804
Dosage,2818
Frequency,2811
Duration (days),0


In [ ]:
dirty_prescriptions_df = inject_data_issues(
    df=prescriptions_df,
    issues_config={
        'missing_values': {
            'columns': ['Dosage', 'Frequency'],  # Exclude Prescription Date here
            'percentage': 0.15
        },
        'duplicates': {
            'percentage': 0.20
        },
        'text_corruption': {
            'columns': ['Frequency'],
            'percentage': 0.15
        }
    }
)

print(dirty_prescriptions_df.sample(10).to_string(index=False))


 Prescription id  Appointment id Patient id  Doctor Id Medication name     Dosage         Frequency  Duration (days)
        94822188        84670500   PT129381   26886866       Albuterol 180mcg/inh     Every 6 hours               30
        98579990        61624200   PT464780    4880720      Lisinopril       40mg            Weekly               30
        41158995        77638485   PT885198   54253968       Metformin      500mg         As needed               30
        53342759        24130487   PT225560   97575200      Sertraline        NaN     Every 8 hours               30
        67526915        36023655   PT337927   41881576       Metformin      500mg         As needed               60
        82165394        32142095   PT757872   90179261      Sertraline      100mg               NaN               60
        81576123        34753497   PT860220   52656520       Albuterol  90mcg/inh Three times daily               90
        36485421        22406589    PT61485   59058825      Omep

In [ ]:
dirty_prescriptions_df.isnull().sum()

,0
Prescription id,0
Appointment id,0
Patient id,0
Doctor Id,0
Medication name,0
Dosage,2695
Frequency,2696
Duration (days),0


In [ ]:
appointments_df.head()

,Appointment id,Patient id,Doctor id,Appointment date,Appointment time,Purpose,Status
0,68675075,PT81957,37309933,2023-07-13,10:07,Diabetes management,Completed
1,16171161,PT561107,76652128,2024-10-06,07:15,Headache,Completed
2,60345092,PT633354,38353840,2024-10-27,10:33,Pain,Completed
3,53346682,PT339587,57477748,2024-09-22,05:44,Cough,Completed
4,99453278,PT387755,35255412,2025-03-17,22:39,Test results,Completed


In [ ]:
appointments_df.dtypes

,0
Appointment id,int64
Patient id,object
Doctor id,int64
Appointment date,object
Appointment time,object
Purpose,object
Status,object


In [ ]:
dirty_appointments_df = inject_data_issues(
    df=appointments_df,
    issues_config={
        'missing_values': {
            'columns': ['Purpose', 'Appointment time', 'Status'],  # Exclude Prescription Date here
            'percentage': 0.15
        },
        'date_format_issues': {
            'columns': ['Appointment date'],
            'percentage': 0.15
        },
        'duplicates': {
            'percentage': 0.18
        },
        'text_corruption': {
            'columns': ['Purpose', 'Status'],
            'percentage': 0.15
        }
    }
)

print(dirty_appointments_df.sample(10).to_string(index=False))


 Appointment id Patient id  Doctor id Appointment date Appointment time      Purpose      Status
       48216519   PT237920   97128107       2024-08-25            19:27    Back pain   Completed
       18185675   PT629998   77812590       2023-06-25            21:22    Allergies         NaN
       79237606   PT965448    6407726       2023-11-09            04:06     Physical Com_:pleted
       30018339   PT310095   18498953       2024-02-24            07:35    Allergies   Completed
       42504718   PT277073   48648428       2023-09-13            04:24    Skin rash         NaN
       33396270   PT854814   24463265       2024-02-04            17:57     Headache   Cancelled
       72160073     PT7145   57477748       2024-11-11            14:17 Al**~lergies         NaN
       69176166   PT101482   48648428       2024-09-08              NaN    Allergies   Completed
         574434   PT535927   10916394       2023-05-19            07:06      Checkup     No-show
        2030905   PT797996   5

In [ ]:
medicines_df.head()

,Medicine id,Name,Brand,Type,Dosage,Stock quantity,Expiry date,Price
0,38738005,Metformin,Glumetza,Biguanide,1000mg,104,2026-12-02,126.65
1,24135250,Atorvastatin,Atorva,Statin,40mg,71,2027-02-15,92.30
2,88786614,Amoxicillin,Amoxil,Antibiotic,250mg,296,2026-10-16,98.69
3,20554285,Albuterol,ProAir,Bronchodilator,180mcg/inh,500,2027-06-20,137.92
4,69651286,Metformin,Glumetza,Biguanide,850mg,73,2026-02-20,145.83


In [ ]:
dirty_medicines_df = inject_data_issues(
    df=medicines_df,
    issues_config={'duplicates': {
            'percentage': 0.18
        },
        'text_corruption': {
            'columns': ['Name', 'Brand'],
            'percentage': 0.15
        }
    }
)

print(dirty_medicines_df.sample(10).to_string(index=False))


 Medicine id            Name     Brand           Type     Dosage  Stock quantity Expiry date  Price
     3352683    Atorvastatin   Torvast         Statin       20mg             192  2026-01-26  78.02
    77742209       Albuterol    ProAir Bronchodilator  90mcg/inh             226  2027-03-15 141.15
    99928652       Albuterol Proventil Bronchodilator 180mcg/inh             261  2026-06-13 146.14
    65333110      Sertraline   Lustral           SSRI       25mg             295  2026-12-23 163.53
    95695601       Metformin  Glumetza      Biguanide     1000mg             407  2027-03-21  46.64
    43601574 Atorvastati|:_n    Atorva         Statin       20mg             385  2025-08-27 152.58
     1990198       Albuterol    ProAir Bronchodilator  90mcg/inh             183  2026-05-31 147.31
    87483746       Metformin  Glumetza      Biguanide     1000mg             417  2026-06-20  84.32
     8535345       Albuterol  Ventolin Bronchodilator  90mcg/inh             223  2027-07-15 111.72


In [ ]:
pharmacy_df.head()

,Pharmacy id,Prescription id,Medicine id,Patient id,Quantity
0,64727811,2455746,14872313,PT81957,2
1,11947946,33763380,52033297,PT561107,4
2,98609541,31089587,51081468,PT561107,1
3,49140147,3198692,86865973,PT633354,4
4,39130973,16900158,86865973,PT339587,4


In [ ]:
dirty_pharmacy_df = inject_data_issues(
    df=pharmacy_df,
    issues_config={
        'missing_values': {
            'columns': ['Quantity'],  # Exclude Prescription Date here
            'percentage': 0.15
        }
    }
)

print(dirty_pharmacy_df.sample(10).to_string(index=False))


 Pharmacy id  Prescription id  Medicine id Patient id  Quantity
    42636135         83976709     20554285   PT351489       4.0
    88515367          5300362     35984591   PT736358       4.0
    36941587         95078650     14872313   PT906055       NaN
    59714565         65222689     28971311   PT385551       5.0
    31080987         97561251     91507728   PT741609       5.0
    62982354         78119733     14872313   PT451473       4.0
    13863511         91833684     79819134   PT906643       5.0
    90182220         98879798     69651286   PT516441       3.0
    25709728         84927534     86865973    PT71539       2.0
    30061017         61221988     86865973   PT252525       2.0


In [ ]:
ambulances_df.head()

,Ambulance id,Ambulance number,Availability,Last service date,Acquisition date
0,598079,NY-AMB-9095,In Maintenance,2024-02-25,2022-05-27
1,443656,CA-AMB-2215,In Maintenance,2022-01-29,2019-01-15
2,685087,OH-AMB-1555,In Maintenance,2024-05-25,2023-07-18
3,608716,MI-AMB-9343,In Maintenance,2022-12-14,2020-08-13
4,5215,PA-AMB-1660,In Maintenance,2024-04-17,2021-11-04


In [ ]:
dirty_ambulances_df = inject_data_issues(
    df=ambulances_df,
    issues_config={
        'missing_values': {
            'columns': ['Availability'],
            'percentage': 0.15
        },
        'date_format_issues': {
            'columns': ['Last service date'],
            'percentage': 0.15
        },
        'duplicates': {
            'percentage': 0.13
        },
        'text_corruption': {
            'columns': ['Availability'],
            'percentage': 0.15
        }
    }
)

print(dirty_ambulances_df.sample(10).to_string(index=False))


 Ambulance id Ambulance number      Availability Last service date Acquisition date
       400228      IL-AMB-6182               NaN        2022-08-28       2019-11-13
       956399      GA-AMB-3856 In (#]Maintenance        2019-01-14       2017-05-22
       882945      FL-AMB-9570    In Maintenance        2021-10-08       2020-09-21
       694067      PA-AMB-8901    In Maintenance        2017.01.20       2016-02-06
       240983      CA-AMB-8871    In Maintenance        2023-11-16       2020-03-31
       379456      IL-AMB-5873    In Maintenance        2021-02-11       2019-01-23
       663583      CA-AMB-7200    In Maintenance        2025-01-11       2016-12-07
       772396      OH-AMB-4933               NaN        2024-11-28       2023-02-02
       505757      GA-AMB-1660    In Maintenance        2022-10-29       2022-05-21
        20645      MI-AMB-6323    In Maintenance        2025-04-01       2024-04-18


In [ ]:
ambulance_logs_df.head()

,Log_id,Ambulance id,Patient id,Pickup location,Dropoff location,Pickup time,Dropoff time,Status
0,56777037,285263,PT285297,"6572 Timothy Lake Apt. 385, South Matthewland","City Medical Center, 77609 Jose Causeway, Port...",2025-03-29 21:37:34,2025-03-29 22:10:34,Completed
1,18201134,598079,PT15493,"1409 Laura Squares Suite 469, Oliverbury","City Medical Center, 9816 Marcus Ramp Apt. 460...",2024-12-26 23:51:08,2024-12-27 00:37:08,Completed
2,91818798,333306,PT948582,"84476 Torres Motorway, East Matthewport","Children's Hospital, 04010 Curtis Fall Suite 2...",2025-02-21 04:17:34,2025-02-21 05:29:34,Completed
3,11474876,919427,PT833002,"499 Hannah Underpass, New Faith","Regional Trauma Center, 853 Brooks Unions Apt....",2025-04-27 17:41:06,2025-04-27 18:22:06,Completed
4,42846755,708082,PT504179,"8842 Walters Court, Vanessaberg","General Hospital, 7118 Bowman Mountains, Coleside",2024-09-18 13:18:07,2024-09-18 14:13:07,Completed


In [ ]:
ambulance_logs_df = ambulance_logs_df.drop(columns=['Dropoff location'])
print(ambulance_logs_df.head())


     Log_id  Ambulance id Patient id  \
0  56777037        285263   PT285297   
1  18201134        598079    PT15493   
2  91818798        333306   PT948582   
3  11474876        919427   PT833002   
4  42846755        708082   PT504179   

                                 Pickup location          Pickup time  \
0  6572 Timothy Lake Apt. 385, South Matthewland  2025-03-29 21:37:34   
1       1409 Laura Squares Suite 469, Oliverbury  2024-12-26 23:51:08   
2        84476 Torres Motorway, East Matthewport  2025-02-21 04:17:34   
3                499 Hannah Underpass, New Faith  2025-04-27 17:41:06   
4                8842 Walters Court, Vanessaberg  2024-09-18 13:18:07   

          Dropoff time     Status  
0  2025-03-29 22:10:34  Completed  
1  2024-12-27 00:37:08  Completed  
2  2025-02-21 05:29:34  Completed  
3  2025-04-27 18:22:06  Completed  
4  2024-09-18 14:13:07  Completed  


In [ ]:
dirty_ambulance_logs_df = inject_data_issues(
    df=ambulance_logs_df,
    issues_config={
        'missing_values': {
            'columns': ['Status'],
            'percentage': 0.15
        },
        'duplicates': {
            'percentage': 0.13
        }
    }
)

print(dirty_ambulance_logs_df.sample(10).to_string(index=False))


  Log_id  Ambulance id Patient id                               Pickup location         Pickup time        Dropoff time    Status
19164686        731361   PT706625        23542 Tracy Tunnel Apt. 142, Levineton 2024-12-20 12:39:21 2024-12-20 13:52:21 Completed
 1861862        836157   PT814344           125 Tammy Estate Apt. 586, Halebury 2024-10-22 16:37:42 2024-10-22 17:15:42 Completed
56922277        547320   PT492686                  3245 Vincent Port, Lake Cody 2024-07-20 07:11:08 2024-07-20 07:49:08 Completed
17079069        637318   PT922579 2380 Bianca Mountains Suite 396, Vanessamouth 2024-09-14 09:07:15 2024-09-14 10:19:15 Completed
76716390        316314   PT562160     17303 Cook Union Apt. 469, East Stephanie 2024-10-05 23:12:45 2024-10-05 23:38:45 Completed
98109522        660604   PT524382          806 Joshua Knoll Suite 257, Carltown 2024-12-12 10:30:40 2024-12-12 11:28:40       NaN
84775148        261571   PT974915           1112 Chad Spurs Apt. 590, Adamshire 2025-03-22

In [ ]:
dirty_ambulance_logs_df.isnull().sum()

,0
Log_id,0
Ambulance id,0
Patient id,0
Pickup location,0
Pickup time,0
Dropoff time,0
Status,102


In [ ]:
dirty_ambulance_logs_df['Status'].unique()

array(['Completed', nan, 'Patient Refused', 'Cancelled', 'Diverted'],
      dtype=object)

In [ ]:
import random
import pandas as pd

def change_unique_value(df, column_name, original_value, new_value, error_percentage):

    df = df.copy()  # Create a copy to avoid modifying the original DataFrame

    # Get the indices of rows containing the original value
    original_value_indices = df.index[df[column_name] == original_value].tolist()

    # Calculate the number of rows to modify
    num_rows_to_modify = int(len(original_value_indices) * error_percentage)

    # Randomly select rows to modify
    rows_to_modify = random.sample(original_value_indices, num_rows_to_modify)

    # Update the selected rows with the new value
    df.loc[rows_to_modify, column_name] = new_value

    return df

In [ ]:
dirty_ambulance_logs_df = change_unique_value(dirty_ambulance_logs_df, 'Status', 'Completed', 'ComPleted', 0.1)
dirty_ambulance_logs_df.head()

,Log_id,Ambulance id,Patient id,Pickup location,Pickup time,Dropoff time,Status
0,56777037,285263,PT285297,"6572 Timothy Lake Apt. 385, South Matthewland",2025-03-29 21:37:34,2025-03-29 22:10:34,Completed
1,18201134,598079,PT15493,"1409 Laura Squares Suite 469, Oliverbury",2024-12-26 23:51:08,2024-12-27 00:37:08,NaN
2,91818798,333306,PT948582,"84476 Torres Motorway, East Matthewport",2025-02-21 04:17:34,2025-02-21 05:29:34,Completed
3,11474876,919427,PT833002,"499 Hannah Underpass, New Faith",2025-04-27 17:41:06,2025-04-27 18:22:06,Completed
4,42846755,708082,PT504179,"8842 Walters Court, Vanessaberg",2024-09-18 13:18:07,2024-09-18 14:13:07,Completed


In [ ]:
rooms_df.head()

,Room id,Room number,Room type,Capacity,Status,Last serviced
0,411052,C665,ICU,1,Needs Inspection,2025-05-12
1,81739,D468,Emergency,1,Ready,2025-05-17
2,560377,C123,CCU,1,Ready,2025-05-17
3,640059,A287,CCU,1,Needs Inspection,2025-05-11
4,330108,B1031,Emergency,1,Ready,2025-05-17


In [ ]:
rooms_df['Status'].unique()

array(['Needs Inspection', 'Ready'], dtype=object)

In [ ]:
dirty_rooms_df = inject_data_issues(
    df=rooms_df,
    issues_config={
        'missing_values': {
            'columns': ['Capacity','Status'],
            'percentage': 0.15
        },
        'date_format_issues': {
            'columns': ['Last serviced'],
            'percentage': 0.14
        },
        'duplicates': {
            'percentage': 0.13
        }
    }
)

dirty_rooms_df.head()
dirty_rooms_df = change_unique_value(dirty_rooms_df, 'Status', 'Needs Inspection', 'Needs inception', 0.13)
dirty_rooms_df.head()


,Room id,Room number,Room type,Capacity,Status,Last serviced
0,411052,C665,ICU,1.0,Needs Inspection,2025-05-12
1,81739,D468,Emergency,1.0,Ready,"May 17, 2025"
2,560377,C123,CCU,1.0,Ready,2025-05-17
3,640059,A287,CCU,NaN,NaN,2025-05-11
4,330108,B1031,Emergency,NaN,NaN,2025.05.17


In [ ]:
room_assignments_df.head()

,Assignment id,Room id,Patient id,Assignment date,End date
0,77193129,136861,PT703598,2023-08-29,2023-09-12
1,26241692,364926,PT830641,2024-11-27,2024-12-02
2,41851909,220565,PT141528,2023-01-09,2023-01-24
3,36556767,423642,PT368851,2024-03-14,2024-03-18
4,4937853,970501,PT583847,2023-02-17,2023-02-24


In [ ]:
dirty_medical_record_medicine_df = medical_record_medicine_df
dirty_medical_record_medicine_df.head()

,MRM id,Record id,Medicine id,Dosage
0,48409720,70626890,14872313,90mcg/inh
1,7946451,17836511,52033297,20mg
2,18732973,17836511,51081468,875mg
3,61233738,77646316,86865973,500mg
4,97168109,46106160,86865973,500mg


In [ ]:
dirty_room_assignments_df = room_assignments_df
dirty_room_assignments_df.head()

,Assignment id,Room id,Patient id,Assignment date,End date
0,77193129,136861,PT703598,2023-08-29,2023-09-12
1,26241692,364926,PT830641,2024-11-27,2024-12-02
2,41851909,220565,PT141528,2023-01-09,2023-01-24
3,36556767,423642,PT368851,2024-03-14,2024-03-18
4,4937853,970501,PT583847,2023-02-17,2023-02-24


In [ ]:
'''
patients, doctors, medical records, prescriptions, appointments, medicine, pharmacy, ambulances, ambulance_logs, rooms, room assignment, medical record medicines
'''

'\npatients, doctors, medical records, prescriptions, appointments, medicine, pharmacy, ambulances, ambulance_logs, rooms, room assignment, medical record medicines\n'

In [ ]:
dirty_billing_df = inject_data_issues(
    df=billing_df,
    issues_config={
        'missing_values': {
            'columns': ['Insurance provider','Payment date'],
            'percentage': 0.15
        },
        'date_format_issues': {
            'columns': ['Payment date'],
            'percentage': 0.14
        },
        'duplicates': {
            'percentage': 0.13
        }
    }
)
dirty_billing_df.head()

,Bill id,Patient id,Appointment id,Total amount,Payment status,Payment date,Insurance provider
0,99501745,PT81957,68675075,937.47,Partially Paid,2023-07-13,NaN
1,6334003,PT561107,16171161,189.98,Partially Paid,"Oct 06, 2024",None
2,76354277,PT633354,60345092,934.61,Partially Paid,2024-10-27,None
3,90002048,PT339587,53346682,952.40,Pending,22-09-2024,UnitedHealthcare
4,30742939,PT387755,99453278,824.87,Partially Paid,NaN,Kaiser Permanente


In [ ]:
dirty_billing_df['Insurance provider'].unique()

array([nan, 'None', 'UnitedHealthcare', 'Kaiser Permanente', 'Cigna',
       'Aetna', 'Blue Cross Blue Shield'], dtype=object)

In [ ]:
dirty_billing_df = change_unique_value(dirty_billing_df, 'Insurance provider', 'Kaiser Permanente', 'Kaiser Permite', 0.13)
dirty_billing_df = change_unique_value(dirty_billing_df, 'Insurance provider', 'Cigna', 'Sigma', 0.14)
dirty_billing_df.head()

,Bill id,Patient id,Appointment id,Total amount,Payment status,Payment date,Insurance provider
0,99501745,PT81957,68675075,937.47,Partially Paid,2023-07-13,NaN
1,6334003,PT561107,16171161,189.98,Partially Paid,"Oct 06, 2024",None
2,76354277,PT633354,60345092,934.61,Partially Paid,2024-10-27,None
3,90002048,PT339587,53346682,952.40,Pending,22-09-2024,UnitedHealthcare
4,30742939,PT387755,99453278,824.87,Partially Paid,NaN,Kaiser Permanente


In [ ]:
dirty_billing_df['Insurance provider'].unique()

array([nan, 'None', 'UnitedHealthcare', 'Kaiser Permanente', 'Cigna',
       'Aetna', 'Blue Cross Blue Shield', 'Kaiser Permite', 'Sigma'],
      dtype=object)

**Tables to be split:**
- patients (3k rows)
- prescriptions (3k rows)
- appointments (3k rows)
- medical records (3k rows)
-medical record medicine
- billing
-pharmacy

patients

In [ ]:
patients_df_extra = patients_df.iloc[:3000].reset_index(drop=True)
patients_df_org = patients_df.iloc[3000:].reset_index(drop=True)

dirty_patients_df_extra = inject_data_issues(
    df=patients_df_extra,
    issues_config={
        'missing_values': {'columns': ['address', 'email', 'blood type'], 'percentage': 0.15},
        'date_format_issues': {'columns': ['dob'], 'percentage': 0.17},
        'duplicates': {'percentage': 0.09},
        'text_corruption': {'columns': ['contact no'], 'percentage': 0.13}
    }
)

print(dirty_patients_df_extra.sample(10).to_string(index=False))

dirty_patients_df_org = inject_data_issues(
    df=patients_df_org,
    issues_config={
        'missing_values': {'columns': ['address', 'email', 'blood type'], 'percentage': 0.15},
        'date_format_issues': {'columns': ['dob'], 'percentage': 0.17},
        'duplicates': {'percentage': 0.09},
        'text_corruption': {'columns': ['contact no'], 'percentage': 0.13}
    }
)

print(dirty_patients_df_extra.sample(10).to_string(index=False))

Patient id              Name          dob  age gender            contact no                                                address                      email blood type
  PT627749      Brian Martin   2015-01-09   10   Male      568.519.5522x886 2056 Elizabeth Camp Apt. 198, East Alexiston, AS 04148       brianmar@example.com         A+
  PT389600         Noah Cruz   1937-11-28   87  Other          766.348.5511   0560 Jennifer Valley Apt. 547, Coleborough, MP 44075      noah_cruz@example.com         B-
  PT710567    Lauren Shepard   2013-09-30   11 Female     (626)872-7170x257    8797 Debbie Crest Suite 151, Whitneyhaven, AR 13542 lauren_shepard@example.com         A-
  PT108954     Michael Mckay   1965-01-29   60  Other       +1-986-325-2518                                                    NaN     michaelmck@example.com         B-
  PT156665       Leah Garcia   2023-10-15    1   Male 001-527-357-8176x3990   7852 Hernandez Shoal Apt. 448, Port Robert, FL 38763    leah.garcia@example.c

In [ ]:
dirty_prescriptions_df.head()

,Prescription id,Appointment id,Patient id,Doctor Id,Medication name,Dosage,Frequency,Duration (days)
0,2455746,68675075,PT81957,37309933,Albuterol,90mcg/inh,Twice daily,90
1,33763380,16171161,PT561107,76652128,Atorvastatin,20mg,NaN,30
2,31089587,16171161,PT561107,76652128,Amoxicillin,875mg,Every 6 hours,10
3,3198692,60345092,PT633354,38353840,Metformin,500mg,Once daily,30
4,16900158,53346682,PT339587,57477748,Metformin,500mg,Every 8 hours,30


prescriptions

In [ ]:
# Extra data split for later use (prescriptions)
extra_patient_ids = patients_df_extra['Patient id'].tolist()
prescriptions_df_extra = prescriptions_df[prescriptions_df['Patient id'].isin(extra_patient_ids)]
prescriptions_df_extra = prescriptions_df_extra.reset_index(drop=True)  # Reset index
prescriptions_df_org = prescriptions_df[~prescriptions_df['Patient id'].isin(extra_patient_ids)]
prescriptions_df_org = prescriptions_df_org.reset_index(drop=True)  # Reset index

In [ ]:
dirty_prescriptions_df_extra = inject_data_issues(
    df=prescriptions_df_extra,
    issues_config={
        'missing_values': {
            'columns': ['Dosage', 'Frequency'],  # Exclude Prescription Date here
            'percentage': 0.15
        },
        'duplicates': {
            'percentage': 0.20
        },
        'text_corruption': {
            'columns': ['Frequency'],
            'percentage': 0.15
        }
    }
)

print(dirty_prescriptions_df.sample(10).to_string(index=False))

dirty_prescriptions_df_org = inject_data_issues(
    df=prescriptions_df_org,
    issues_config={
        'missing_values': {
            'columns': ['Dosage', 'Frequency'],  # Exclude Prescription Date here
            'percentage': 0.15
        },
        'duplicates': {
            'percentage': 0.20
        },
        'text_corruption': {
            'columns': ['Frequency'],
            'percentage': 0.15
        }
    }
)

print(dirty_prescriptions_df.sample(10).to_string(index=False))

 Prescription id  Appointment id Patient id  Doctor Id Medication name    Dosage     Frequency  Duration (days)
        39543761        73340341   PT493577   84448772     Amoxicillin     250mg   Twice daily               10
        15593523        58835327   PT503054   94058591       Metformin     500mg Every 8 hours               30
        33292385        70802074   PT985936   25596377     Amoxicillin     500mg     As needed                7
        77047019        61037979   PT110878   57453492    Atorvastatin       NaN Onc;`,e daily               30
        43383340        14268193   PT788754   35661451      Sertraline     100mg           NaN               30
        59855262        81460127    PT51691   41881576       Albuterol 90mcg/inh        Weekly               30
        16676232        78317990    PT82617   61204216       Metformin     500mg    Once daily               30
          208702        57289067    PT69442   24463265     Amoxicillin       NaN    Once daily          

In [ ]:
dirty_prescriptions_df_org.shape

(14284, 8)

appointments

In [ ]:
dirty_appointments_df.head()

,Appointment id,Patient id,Doctor id,Appointment date,Appointment time,Purpose,Status
0,68675075,PT81957,37309933,2023-07-13,10:07,Diabetes management,C&ompleted
1,16171161,PT561107,76652128,10/06/2024,07:15,Head:ache,Completed
2,60345092,PT633354,38353840,2024-10-27,NaN,Pain{|,Completed
3,53346682,PT339587,57477748,2024-09-22,05:44,Cough,C<]ompleted
4,99453278,PT387755,35255412,2025-03-17,22:39,Test results,Completed


In [ ]:
# Extra data split for later use (appointments)

appointments_df_extra = appointments_df[appointments_df['Patient id'].isin(extra_patient_ids)]
appointments_df_extra = appointments_df_extra.reset_index(drop=True)  # Reset index
appointments_df_org = appointments_df[~appointments_df['Patient id'].isin(extra_patient_ids)]
appointments_df_org = appointments_df_org.reset_index(drop=True)  # Reset index

In [ ]:
dirty_appointments_df_extra = inject_data_issues(
    df=appointments_df_extra,
    issues_config={
        'missing_values': {
            'columns': ['Purpose', 'Appointment time', 'Status'],  # Exclude Prescription Date here
            'percentage': 0.15
        },
        'date_format_issues': {
            'columns': ['Appointment date'],
            'percentage': 0.15
        },
        'duplicates': {
            'percentage': 0.18
        },
        'text_corruption': {
            'columns': ['Purpose', 'Status'],
            'percentage': 0.15
        }
    }
)

print(dirty_appointments_df.sample(10).to_string(index=False))

dirty_appointments_df_org = inject_data_issues(
    df=appointments_df_org,
    issues_config={
        'missing_values': {
            'columns': ['Purpose', 'Appointment time', 'Status'],  # Exclude Prescription Date here
            'percentage': 0.15
        },
        'date_format_issues': {
            'columns': ['Appointment date'],
            'percentage': 0.15
        },
        'duplicates': {
            'percentage': 0.18
        },
        'text_corruption': {
            'columns': ['Purpose', 'Status'],
            'percentage': 0.15
        }
    }
)

print(dirty_appointments_df.sample(10).to_string(index=False))

 Appointment id Patient id  Doctor id Appointment date Appointment time     Purpose       Status
       99536814    PT36184    4350834       2023-06-21            23:10         NaN    Completed
       36969982   PT777903   46303541       2023-10-31            11:04         NaN Compl>?<eted
       91152627   PT463869    4392467       2024-11-08            16:43 Annual exam    Completed
       37495745   PT910018   33634906       2023-11-18            14:04    Physical      No-show
        8994364   PT795422    9180682       2023-05-27              NaN    Headache          NaN
        7271070   PT798331   95947184       2025-01-12            22:08         NaN    Completed
       51518893   PT872284   66896983       2025-03-14            20:30   Skin rash   Co(mpleted
        2775734   PT531499   10842512       2024-04-14            10:01  Physi_[cal    Completed
       69091653   PT245078   23768035       2024.08.06            11:31        Pain    Completed
       21062911   PT584687    

In [ ]:
dirty_appointments_df_org.shape

(9418, 7)

In [ ]:
dirty_appointments_df_extra.head()

,Appointment id,Patient id,Doctor id,Appointment date,Appointment time,Purpose,Status
0,60345092,PT633354,38353840,2024-10-27,10:33,Pain,Completed
1,28776469,PT17871,14200954,2024-01-22,04:45,Annual exam,Completed
2,6100773,PT988924,22655541,2023-06-02,05:27,He:])adache,Completed
3,5283264,PT235181,52013905,2023-11-20,NaN,Back pain,Completed
4,57846488,PT140037,96655296,2024-01-04,NaN,NaN,Completed


medical records

In [ ]:
dirty_medical_records_df.isnull().sum()

,0
Record id,0
Appointment id,0
Patient id,0
Doctor id,0
Diagnosis,2087
Treatment,2086


In [ ]:
# Extra data split for later use (medical records)
medical_records_df_extra = medical_records_df[medical_records_df['Patient id'].isin(extra_patient_ids)]
medical_records_df_extra = medical_records_df_extra.reset_index(drop=True)  # Reset index
medical_records_df_org = medical_records_df[~medical_records_df['Patient id'].isin(extra_patient_ids)]
medical_records_df_org = medical_records_df_org.reset_index(drop=True)  # Reset index

In [ ]:
medical_records_df_extra.head()

,Record id,Appointment id,Patient id,Doctor id,Diagnosis,Treatment
0,77646316,60345092,PT633354,38353840,Acute Bronchitis (J20.9),Surgery scheduled
1,91440445,28776469,PT17871,14200954,Back Pain (M54.5),Monitoring recommended
2,59422702,6100773,PT988924,22655541,Type 2 Diabetes (E11.9),Prescribed Metformin 500mg twice daily
3,4267822,5283264,PT235181,52013905,Type 2 Diabetes (E11.9),Prescribed Metformin 500mg twice daily
4,15855878,57846488,PT140037,96655296,Major Depressive Disorder (F32.9),Observation only


In [ ]:
dirty_medical_records_df_extra = inject_data_issues(
    df=medical_records_df_extra,
    issues_config={
        'missing_values': {'columns': ['Treatment', 'Diagnosis'], 'percentage': 0.2},
        'duplicates': {'percentage': 0.05},
        'text_corruption': {'columns': ['Diagnosis', 'Treatment'], 'percentage': 0.1}
    }
)

print(dirty_medical_records_df.sample(10).to_string(index=False))

dirty_medical_records_df_org = inject_data_issues(
    df=medical_records_df_org,
    issues_config={
        'missing_values': {'columns': ['Treatment', 'Diagnosis'], 'percentage': 0.2},
        'duplicates': {'percentage': 0.05},
        'text_corruption': {'columns': ['Diagnosis', 'Treatment'], 'percentage': 0.1}
    }
)

print(dirty_medical_records_df.sample(10).to_string(index=False))

 Record id  Appointment id Patient id  Doctor id               Diagnosis                        Treatment
  53606299        89786844   PT510092    7770890        Asthma (J45.909)                 Observation only
   4864182        80332012   PT103859   38163960                     NaN           Referral to specialist
  40021035        76293289   PT235484   78821418                     NaN           Monitoring recommended
  82448969        27485480   PT436062   53912690       Back Pain (M54.5)                 Observation only
  82667931        83977430   PT386648   31765842      Migraine (G43.909)            Prescribed medication
  98128688        71850396   PT522577   11694555        Asthma (J45.909)                Surgery scheduled
  45680360        85307513   PT708168   18277588        Asthma (J45.909)           Lifestyle modification
  68329632        91372490   PT751805   44096183        Asthma (J45.909)           IV fluids administered
  65640049        74718711   PT136830    46112

In [ ]:
dirty_medical_records_df_extra.head()

,Record id,Appointment id,Patient id,Doctor id,Diagnosis,Treatment
0,77646316,60345092,PT633354,38353840,Acute Bronchitis (J20.9),Surgery scheduled
1,91440445,28776469,PT17871,14200954,Back Pain (M54.5),Monitoring recommended
2,59422702,6100773,PT988924,22655541,Type 2 Diabetes (E11.9),Prescribed Metformin 500mg twice daily
3,4267822,5283264,PT235181,52013905,Type 2 Diabetes (E11.9),Prescribed Metformin 500mg twice daily
4,15855878,57846488,PT140037,96655296,Major Depressive Disorder (F32.9),Observation only


medical record medicine

In [ ]:
# Extract extra Record ids
extra_record_ids = medical_records_df_extra['Record id'].tolist()

# Filter medical_record_medicine_df using MRM id (which maps to Record id)
medical_record_medicine_df_extra = medical_record_medicine_df[medical_record_medicine_df['Record id'].isin(extra_record_ids)]
medical_record_medicine_df_org = medical_record_medicine_df[~medical_record_medicine_df['Record id'].isin(extra_record_ids)]

In [ ]:
medical_record_medicine_df_extra.head()

,MRM id,Record id,Medicine id,Dosage
3,61233738,77646316,86865973,500mg
28,83307020,91440445,2986098,40mg
29,58237102,59422702,51081468,875mg
30,35882896,59422702,96327970,10mg
33,74582893,4267822,51081468,875mg


In [ ]:
dirty_medical_record_medicine_df_extra = medical_record_medicine_df_extra
dirty_medical_record_medicine_df_org = medical_record_medicine_df_org

pharmacy

In [ ]:
# Extra data split for later use (medical records)
pharmacy_df_extra = pharmacy_df[pharmacy_df['Patient id'].isin(extra_patient_ids)]
pharmacy_df_extra = pharmacy_df_extra.reset_index(drop=True)  # Reset index
pharmacy_df_org = pharmacy_df[~pharmacy_df['Patient id'].isin(extra_patient_ids)]
pharmacy_df_org = pharmacy_df_org.reset_index(drop=True)  # Reset index

In [ ]:
dirty_pharmacy_df_extra = inject_data_issues(
    df=pharmacy_df_extra,
    issues_config={
        'missing_values': {
            'columns': ['Quantity'],  # Exclude Prescription Date here
            'percentage': 0.15
        }
    }
)

print(dirty_pharmacy_df.sample(10).to_string(index=False))

dirty_pharmacy_df_org = inject_data_issues(
    df=pharmacy_df_org,
    issues_config={
        'missing_values': {
            'columns': ['Quantity'],  # Exclude Prescription Date here
            'percentage': 0.15
        }
    }
)

print(dirty_pharmacy_df.sample(10).to_string(index=False))


 Pharmacy id  Prescription id  Medicine id Patient id  Quantity
    88515367          5300362     35984591   PT736358       4.0
     8293793         36779664      2098773    PT57281       NaN
    90198294         95843763      2986098    PT60823       2.0
    54512918         29146892     86865973   PT780409       3.0
    23746486         87385130     79819134   PT734583       1.0
    45778056         49157740     88786614   PT242517       2.0
     7725450         84007865     88786614   PT666750       2.0
    19784029         65239167     20554285   PT322947       2.0
    87739843         18759318     79819134   PT771719       5.0
    67539953         53454873     91507728   PT741103       4.0
 Pharmacy id  Prescription id  Medicine id Patient id  Quantity
    99019683         86930940     51081468   PT975984       5.0
     1208897         16141768     91507728   PT761204       5.0
     6233274          4308409     38738005   PT164821       1.0
    84550217          9176854      71562

billing

In [ ]:
# Extra data split for later use (billing)

extra_patient_ids_1 = appointments_df_extra['Patient id'].tolist()  # Already done previously
extra_appointment_ids = appointments_df_extra['Appointment id'].tolist()
billing_df_extra = billing_df[
    (billing_df['Patient id'].isin(extra_patient_ids_1)) &
    (billing_df['Appointment id'].isin(extra_appointment_ids))
]
billing_df_extra = billing_df_extra.reset_index(drop=True)  # Reset index

In [ ]:
billing_df_org = billing_df[
    ~(
        (billing_df['Patient id'].isin(extra_patient_ids_1)) &
        (billing_df['Appointment id'].isin(extra_appointment_ids))
    )
]
billing_df_org = billing_df_org.reset_index(drop=True)  # Reset index

In [ ]:
dirty_billing_df_extra = inject_data_issues(
    df=billing_df_extra,
    issues_config={
        'missing_values': {
            'columns': ['Insurance provider','Payment date'],
            'percentage': 0.15
        },
        'date_format_issues': {
            'columns': ['Payment date'],
            'percentage': 0.14
        },
        'duplicates': {
            'percentage': 0.13
        }
    }
)
dirty_billing_df_extra.head()

,Bill id,Patient id,Appointment id,Total amount,Payment status,Payment date,Insurance provider
0,76354277,PT633354,60345092,934.61,Partially Paid,NaN,NaN
1,15670355,PT17871,28776469,736.60,Paid,2024-01-22,UnitedHealthcare
2,16433776,PT988924,6100773,830.13,Denied,2023-06-02,Blue Cross Blue Shield
3,55064377,PT235181,5283264,169.01,Paid,20-11-2023,Blue Cross Blue Shield
4,49051636,PT140037,57846488,964.88,Paid,2024-01-04,Blue Cross Blue Shield


In [ ]:
dirty_billing_df_org = inject_data_issues(
    df=billing_df_org,
    issues_config={
        'missing_values': {
            'columns': ['Insurance provider','Payment date'],
            'percentage': 0.15
        },
        'date_format_issues': {
            'columns': ['Payment date'],
            'percentage': 0.14
        },
        'duplicates': {
            'percentage': 0.13
        }
    }
)
dirty_billing_df_org.head()

,Bill id,Patient id,Appointment id,Total amount,Payment status,Payment date,Insurance provider
0,99501745,PT81957,68675075,937.47,Partially Paid,2023-07-13,None
1,6334003,PT561107,16171161,189.98,Partially Paid,2024-10-06,None
2,90002048,PT339587,53346682,952.40,Pending,2024-09-22,NaN
3,30742939,PT387755,99453278,824.87,Partially Paid,2025-03-17,Kaiser Permanente
4,51309299,PT311757,21793989,723.50,Paid,2023-07-11,Cigna


In [ ]:
dirty_billing_df_extra.shape

(2280, 7)

In [ ]:
dirty_billing_df_org.shape

(9019, 7)

In [ ]:
dirty_medical_record_medicine_df_extra.shape

(3038, 4)

In [ ]:
dirty_medical_record_medicine_df_org.shape

(11904, 4)

In [ ]:
# For original dirty data tables

from google.colab import files
dirty_dfs = {
    'ambulances_org': dirty_ambulances_df,
    'ambulance_logs_org': dirty_ambulance_logs_df,
    'rooms_org': dirty_rooms_df,
    'room_assignments_org': dirty_room_assignments_df,
    'medical_record_medicines_org': dirty_medical_record_medicine_df_org,
    'billing_org': dirty_billing_df_org
}

for table_name, df in dirty_dfs.items():
    csv_file_name = f"{table_name}.csv"
    df.to_csv(csv_file_name, index=False)
    files.download(csv_file_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

mrm, pharmacy

In [ ]:
# For original dirty data tables

from google.colab import files
dirty_dfs = {
    'patients_org': dirty_patients_df_org,
    'doctors_org': dirty_doctors_df,
    'medical_records_org': dirty_medical_records_df_org,
    'prescriptions_org': dirty_prescriptions_df_org,
    'appointments_org': dirty_appointments_df_org,
    'medicines_org': dirty_medicines_df,
}

for table_name, df in dirty_dfs.items():
    csv_file_name = f"{table_name}.csv"
    df.to_csv(csv_file_name, index=False)
    files.download(csv_file_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# For original dirty data tables

from google.colab import files
dirty_dfs = {
    'patients_extra': dirty_patients_df_extra,
    'medical_records_extra': dirty_medical_records_df_extra,
    'prescriptions_extra': dirty_prescriptions_df_extra,
    'appointments_extra': dirty_appointments_df_extra,
    'billing_extra': dirty_billing_df_extra,
    'pharmacy_extra': dirty_pharmacy_df_extra,
}

for table_name, df in dirty_dfs.items():
    csv_file_name = f"{table_name}.csv"
    df.to_csv(csv_file_name, index=False)
    files.download(csv_file_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#For medical record medicine data tables

from google.colab import files
dirty_dfs = {
    'dirty_medical_record_medicine_df_org': dirty_medical_record_medicine_df_org,
    'dirty_medical_record_medicine_df_extra': dirty_medical_record_medicine_df_extra,
}

for table_name, df in dirty_dfs.items():
    csv_file_name = f"{table_name}.csv"
    df.to_csv(csv_file_name, index=False)
    files.download(csv_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
dirty_dfs = {
    'pharmacy_org': dirty_pharmacy_df_org,
}
for table_name, df in dirty_dfs.items():
    csv_file_name = f"{table_name}.csv"
    df.to_csv(csv_file_name, index=False)
    files.download(csv_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>